<a href="https://colab.research.google.com/github/kweenkeen/PyTorchPractice/blob/master/PyTorchPractice3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL

For this tutorial, we will be finetuning a pre-trained Mask R-CNN model in the Penn-Fudan Database for Pedestrial Detection and Segmentation. It contains 170 images with 345 instances of pedestrians, and we will use it to illustrate how to use the new features in torchvision in order to train an instance segmentation model on a custom dataset.

### Defining the Dataset

The reference scripts for training object detection, instance segmentation and person keypoint detection allows for easily supporting adding new custom datasets. The dataset should inherit from the standard `torch.utils.data.Dataset` class, and implement `__len__` and `__getitem__`.

The only specificity that we require is that the dataset `__getitem__` should return:

*   image: a PIL Image of size **`(H, W)`**
*   target: a dict containing the following fields
  * **`boxes`** (FloatTensor[N,4]): the coordinates of the **`N`**  bounding boxes in **`[x0, y0, x1, y1]`** format, ranging from **`0`** to **`W`** and **`0`** to **`H`**
  * **`labels (Int64Tensor[N])`**: the label for each bounding box. **`0`** represents always the background class
  * **`image_id (Int64Tensor[1]`**: an image identifier. It should be unique between all the images in the dataset and is used during evaluation.
  * **`area (Tensor[N])`**: The area of the bounding box. This is used during evaluation with the COCO metric, to separate the metric scores between small, medium, and large boxes.
  * **`iscrowd (UInt8Tensor[N])`**: instances wih iscrowd=True will be ignored during evaluation



In [39]:
pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

  Cloning https://github.com/gautamchitnis/cocoapi.git (to revision cocodataset-master) to /tmp/pip-req-build-rnn1qn0b
  Running command git clone -q https://github.com/gautamchitnis/cocoapi.git /tmp/pip-req-build-rnn1qn0b
  Running command git checkout -b cocodataset-master --track origin/cocodataset-master
  Switched to a new branch 'cocodataset-master'
  Branch 'cocodataset-master' set up to track remote branch 'cocodataset-master' from 'origin'.
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266697 sha256=95ad7257c0aa15fe6203a6209a6264165bef856c152d3c2f7c591ebdc3ce893a
  Stored in directory: /tmp/pip-ephem-wheel-cache-fir0m3f5/wheels/49/ca/2b/1b99c52bb9e7a9804cd60d66243ec70c9f15977795793d2646
Successfully built pycocotools


In [42]:
from torchvision.datasets.utils import download_file_from_google_drive
torchvision.datasets.utils.extract_archive('PennFudanPed.zip')

FileNotFoundError: ignored

In [43]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset(object):
  def __init__(self, root, transforms):
    self.root = root
    self.transforms = transforms
    # load all image files, sorting them to
    # ensure that they are aligned
    self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
    self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

  def __getitem__(self,idx):
    # load images and masks
    img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
    mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
    img = Image.open(img_path).convert("RGB")
    # note that wehaven't converted the mask to RGB
    # because each color corresponds to a different instance
    # with 0 being background
    mask = Image.open(mask_path)
    # convert the PIL Image into a numpy array
    mask = np.array(mask)
    # instances are encoded as different colors
    obj_ids = np.unique(mask)
    # first id is the background, so remove it
    obj_ids = obj_ids[1:]

    # split the color-encoded mask into a set
    # of binary masks
    masks = mask == obj_ids[:, None, None]

    # get bounding box coordinates for each mask
    num_objs = len(obj_ids)
    boxes = []
    for i in range(num_objs):
      pos = np.where(masks[i])
      xmin = np.min(pos[1])
      xmax = np.max(pos[1])
      ymin = np.min(pos[0])
      ymax = np.max(pos[0])
      boxes.append([xmin, ymin, xmax, ymax])

    # convert everything into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype = torch.float32)
    # there is only one class
    labels = torch.ones((num_objs), dtype=torch.float32)
    # there is only one class
    labels = torch.ones((num_objs,), dtype=torch.int64)
    masks = torch.as_tensor(masks, dtype=torch.uint8)

    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:,2] - boxes[:, 0])
    # suppose all instances are now crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["masks"] = masks
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd

    if self.transforms is not None:
      img, target = self.transforms(img, target)
    return img, target

    def __len__(self):
      return len(self.imgs)

### Defining your model

In this tutorial, we will be using Mask R-CNN, which is based on top of Faster R-CNN. Faster R-CNN is a model that predicts both bounding boxes and class scores for potential objects in the image.

There are two common situations where one might want to modify one of the available models in torchvision modelzoo. The first is when we want to start from a pre-trained model, and just finetune the last layer. The other is when we want to replace the backbone of the model with a different one (for faster predictions, for example).

#### 1. Finetuning from a pretrained model

Let's suppose that you want to start from a model pre-trained on COCO and want to finetune it for your particular classes. Here is a possible way of doing it:

In [46]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

NameError: ignored

In [45]:
# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# replace the classifier with a new one that has num_classes
# which is user-defined
num_classes = 2 # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

### 2 - Modifying the model to add a different backbone

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect ratios.
# We have a Tuple[Tuple[int]] because each feature map could
# potentially have different sizes and aspect ratios.
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                  aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will use to perform
# the region of interest cropping, as well as the size of the crop after
# rescaling.
# If your backbone returns a Tensor, featmap_names is expected to be [0].
# More generally, the backbone should return an OrderedDict[Tensor], and
# in featmap_names you can choose which feature maps to use.

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

#### An Instance Segmentation model for PennFudan Dataset

In our case, we want to fine-tune from a pre-trained model, given hat our dataset is very small, so we will be following approach number 1.

Here we want to also compute the instance segmentation masks, so we will be using Mask R-CNN:

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
  # load an instance segmentation model pre-trained on COCO
  model = torchdivision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_head.box_predictor = FastRCNNPredictor(in_features, num_classes)

  # now get the number of input features for the mask classifier
  in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
  hidden_layer = 256
  # and replace the mask predictor with a new one
  model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                     hidden_layer,
                                                     num_classes)
  return model

### Putting everything together

In ```references/detection/```, we have a number of helper functions to simplify training and evaluating detection models. Here, we will use ```references/detection/engine.py```, ```references/detection/utils.py``` and ```references/detection/transforms.py```.

Some helper functions for data augmentation/transformations:


In [ ]:
import torchvision.transforms as T
def get_transform(train):
  transforms = []
  transforms.append(T.ToTensor())
  if train:
    transforms.append(T.RandomHorizontalFlip(0.5))
  return T.Compose(transforms)

### Testing ```forward()``` method (Optional)
Before iterating over the dataset, it's good to see what the model expects during training and inference time on sample data.

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets) # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  #Returns predictions

FileNotFoundError: ignored

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)